In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('create_session').getOrCreate()

# **Tables**

Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame, and dataframe.write().csv("path") to write to a CSV file. Function option() can be used to customize the behavior of reading or writing, such as controlling behavior of the header, delimiter character, character set, and so on.

In [3]:
df1=spark.read.option("header", "true").option("inferSchema", "true").csv('D:/LocalSetups/Jupyter/Data/matches.csv')
df2=spark.read.option("header", "true").option("inferSchema", "true").csv('D:/LocalSetups/Jupyter/Data/deliveries.csv')

We can create a temporary view on DataFrame/Dataset by using createOrReplaceTempView() and using SQL to select and manipulate the data

In [ ]:
df1.createOrReplaceTempView("matches")
df2.createOrReplaceTempView("deliveries")

In [6]:
spark.sql("""
SELECT *
FROM matches
LIMIT 1
""").show()

+-------+------+---------+----------+--------------+-------------------+-------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+--------------------+-----------+---------+--------------------+
|     id|season|     city|      date|         team1|              team2|        toss_winner|toss_decision|result|dl_applied|             winner|win_by_runs|win_by_wickets|player_of_match|               venue|    umpire1|  umpire2|             umpire3|
+-------+------+---------+----------+--------------+-------------------+-------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+--------------------+-----------+---------+--------------------+
|1370353|  2023|Ahmedabad|2023/05/29|Gujarat Titans|Chennai Super Kings|Chennai Super Kings|        field|   D/L|         1|Chennai Super Kings|          0|             5|      DP Conway|Narendra Modi Sta...|Nitin Menon|RJ Tucker|KN Ananthapadm

In [32]:
spark.sql("""
SELECT *
FROM deliveries
LIMIT 1
""").show()

+--------+------+----------+--------------------+-------+----+--------------+-------------------+-------+------------+---------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+------------+
|match_id|season|start_date|               venue|innings|ball|  batting_team|       bowling_team|striker| non_striker|   bowler|runs_off_bat|extras|wides|noballs|byes|legbyes|penalty|wicket_type|player_dismissed|other_wicket_type|other_player_dismissed|cricsheet_id|
+--------+------+----------+--------------------+-------+----+--------------+-------------------+-------+------------+---------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+------------+
| 1370353|  2023|2023-05-29|Narendra Modi Sta...|      1| 0.1|Gujarat Titans|Chennai Super Kings|WP Saha|Shubman Gill|DL Chahar|           0|     0| NULL|   NULL|NULL|   NULL|   NULL|       NULL|    

In [8]:
# display function gives you the columns name and their data types.
display(
spark.sql("""
SELECT *
FROM deliveries
LIMIT 1
""")
)

DataFrame[match_id: int, season: string, start_date: date, venue: string, innings: int, ball: double, batting_team: string, bowling_team: string, striker: string, non_striker: string, bowler: string, runs_off_bat: int, extras: int, wides: double, noballs: double, byes: double, legbyes: double, penalty: double, wicket_type: string, player_dismissed: string, other_wicket_type: string, other_player_dismissed: string, cricsheet_id: int]

# **Questions**

***1. What is the percentage of wins by a team which has won the toss in Bangalore stadium?***

In [7]:
spark.sql( """
SELECT toss_winner AS team,
    ROUND(100*SUM(CASE WHEN toss_winner=winner THEN 1 ELSE 0 END )/COUNT(*),2)  AS win_percentage
FROM matches
WHERE city='Bangalore'
GROUP BY toss_winner
""").show()

+--------------------+--------------+
|                team|win_percentage|
+--------------------+--------------+
| Sunrisers Hyderabad|          50.0|
| Chennai Super Kings|          50.0|
|     Deccan Chargers|          50.0|
|Kochi Tuskers Kerala|           0.0|
|    Rajasthan Royals|         66.67|
|       Gujarat Lions|          50.0|
|Royal Challengers...|         47.83|
|Kolkata Knight Ri...|         100.0|
|     Kings XI Punjab|         42.86|
|       Pune Warriors|           0.0|
|    Delhi Daredevils|          50.0|
|      Mumbai Indians|          75.0|
+--------------------+--------------+



***2. Display total matches played, total matches won and total matches lost by team along with its team name***

In [14]:
spark.sql("""
WITH base_data AS (
    SELECT team1 AS team_name,
        COUNT(DISTINCT id) AS total_matches,
        COUNT(DISTINCT CASE WHEN result <> 'no result' AND team1 = winner THEN id ELSE NULL END) AS total_matches_won,
        COUNT(DISTINCT CASE WHEN result <> 'no result' AND team1 <> winner THEN id ELSE NULL END) AS total_matches_lost
    FROM matches
    GROUP BY 1

    UNION ALL

    SELECT team2 AS team_name,
        COUNT(DISTINCT id) AS total_matches,
        COUNT(DISTINCT CASE WHEN result <> 'no result' AND team2 = winner THEN id ELSE NULL END) AS total_matches_won,
        COUNT(DISTINCT CASE WHEN result <> 'no result' AND team2 <> winner THEN id ELSE NULL END) AS total_matches_lost
    FROM matches
    GROUP BY 1
)

SELECT team_name AS `Team Name`,
    SUM(total_matches) AS `Total Matches`,
    SUM(total_matches_won) AS `Total Matches Won`,
    SUM(total_matches_lost) AS `Total Matches Lost`
FROM base_data
GROUP BY 1
ORDER BY 3 DESC;
""").show()

+--------------------+-------------+-----------------+------------------+
|           Team Name|Total Matches|Total Matches Won|Total Matches Lost|
+--------------------+-------------+-----------------+------------------+
|      Mumbai Indians|          247|              138|               105|
| Chennai Super Kings|          224|              131|                91|
|Kolkata Knight Ri...|          237|              119|               114|
|Royal Challengers...|          240|              114|               120|
|    Rajasthan Royals|          206|              101|               100|
|     Kings XI Punjab|          190|               85|               101|
| Sunrisers Hyderabad|          166|               78|                84|
|    Delhi Daredevils|          161|               67|                91|
|      Delhi Capitals|           77|               38|                36|
|     Deccan Chargers|           75|               29|                46|
|      Gujarat Titans|           33|  

In [17]:
spark.sql(
    '''SELECT *
       FROM matches
       LIMIT 1
       '''
).show()

+-------+------+---------+----------+--------------+-------------------+-------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+--------------------+-----------+---------+--------------------+
|     id|season|     city|      date|         team1|              team2|        toss_winner|toss_decision|result|dl_applied|             winner|win_by_runs|win_by_wickets|player_of_match|               venue|    umpire1|  umpire2|             umpire3|
+-------+------+---------+----------+--------------+-------------------+-------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+--------------------+-----------+---------+--------------------+
|1370353|  2023|Ahmedabad|2023/05/29|Gujarat Titans|Chennai Super Kings|Chennai Super Kings|        field|   D/L|         1|Chennai Super Kings|          0|             5|      DP Conway|Narendra Modi Sta...|Nitin Menon|RJ Tucker|KN Ananthapadm

 ***3. Top 10 Batsman based on batting average***

In [10]:
spark.sql("""
WITH batsman_runs AS (
    	SELECT striker AS player,
             SUM(runs_off_bat) AS total_runs
    	FROM deliveries
    	GROUP BY 1
         ),
    batsman_dismissed AS (
    	SELECT player_dismissed AS player,
             COUNT(player_dismissed) AS total_dismissed
    	FROM deliveries
        WHERE wicket_type <> 'retired hurt' 
    	GROUP BY player_dismissed )
    
SELECT A.player, 
    ROUND( total_runs/total_dismissed ,2) AS batting_avg
FROM batsman_runs A
JOIN batsman_dismissed B ON A.player=B.player
ORDER BY batting_avg DESC
LIMIT 10 
""").show()

+---------------+-----------+
|         player|batting_avg|
+---------------+-----------+
| Vivrant Sharma|       69.0|
|     MN van Wyk|      55.67|
|        C Green|      50.22|
|      DP Conway|      48.63|
|B Sai Sudharsan|      46.09|
|       KL Rahul|      45.81|
|       AC Voges|      45.25|
|        HM Amla|      44.38|
|  Iqbal Abdulla|       44.0|
|      DA Warner|      41.02|
+---------------+-----------+



***4. Top 10 Batsman based on batting strike rate to played minimum 200 balls***

In [65]:
spark.sql("""
SELECT striker,
    100*SUM(runs_off_bat)/COUNT(*) AS batting_strike_rate
FROM deliveries
GROUP BY striker
HAVING COUNT(*)>200
ORDER BY batting_strike_rate DESC
LIMIT 10
""").show()

+--------------+-------------------+
|       striker|batting_strike_rate|
+--------------+-------------------+
|      TH David|  167.8714859437751|
|    AD Russell| 164.91994177583697|
|     H Klaasen|            160.625|
|   Rashid Khan|  158.2142857142857|
|       C Green| 157.49128919860627|
|LS Livingstone| 157.11574952561668|
|     JM Sharma| 153.38983050847457|
|    GJ Maxwell|  151.4476614699332|
|     SP Narine| 151.15606936416185|
|      N Pooran|  149.9409681227863|
+--------------+-------------------+



***5. Top 10 batsman based on highest score***

In [59]:
spark.sql("""
SELECT season, 
    striker,
    SUM(runs_off_bat) AS highest_score
FROM deliveries
GROUP BY match_id,season,striker
ORDER BY highest_score DESC
LIMIT 10
""").show()

+-------+--------------+-------------+
| season|       striker|highest_score|
+-------+--------------+-------------+
|   2013|      CH Gayle|          175|
|2007/08|   BB McCullum|          158|
|   2022|     Q de Kock|          140|
|   2015|AB de Villiers|          133|
|2020/21|      KL Rahul|          132|
|   2016|AB de Villiers|          129|
|   2023|  Shubman Gill|          129|
|   2018|       RR Pant|          128|
|   2012|      CH Gayle|          128|
|2009/10|       M Vijay|          127|
+-------+--------------+-------------+



 ***6. Orange Cap Holder***

In [60]:
spark.sql("""
WITH batsman_run AS (
    SELECT season,
        striker,
        SUM(runs_off_bat) AS total_runs
    FROM deliveries
    GROUP BY 1,2
    )
SELECT season,
    striker,
    total_runs
FROM(
    SELECT *, 
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY total_runs DESC) AS rnk
    FROM batsman_run
    ) 
WHERE rnk=1 
""").show()

+-------+-------------+----------+
| season|      striker|total_runs|
+-------+-------------+----------+
|2007/08|     SE Marsh|       616|
|   2009|    ML Hayden|       572|
|2009/10| SR Tendulkar|       618|
|   2011|     CH Gayle|       608|
|   2012|     CH Gayle|       733|
|   2013|   MEK Hussey|       733|
|   2014|   RV Uthappa|       660|
|   2015|    DA Warner|       562|
|   2016|      V Kohli|       973|
|   2017|    DA Warner|       641|
|   2018|KS Williamson|       735|
|   2019|    DA Warner|       692|
|2020/21|     KL Rahul|       676|
|   2021|   RD Gaikwad|       635|
|   2022|   JC Buttler|       863|
|   2023| Shubman Gill|       890|
+-------+-------------+----------+



 ***7. Purple Cap Holder***

In [10]:
spark.sql( """
WITH cte AS (
	SELECT season,
        bowler,
        COUNT(player_dismissed) AS total_wicket,COUNT(CASE WHEN wicket_type = 'bowled' THEN 1 ELSE NULL END) AS bowled_wicket,
        COUNT(CASE WHEN wicket_type = 'stumped' THEN 1 ELSE NULL END) AS stumped_wicket,  COUNT(CASE WHEN wicket_type = 'lbw' THEN 1 ELSE NULL END) AS lbw_wicket,
        COUNT(CASE WHEN wicket_type = 'caught and bowled' THEN 1 ELSE NULL END) AS caught_and_bowled_wicket,
        COUNT(CASE WHEN wicket_type = 'hit wicket' THEN 1 ELSE NULL END) AS hit_wicket,
        COUNT(CASE WHEN wicket_type = 'caught' THEN 1 ELSE NULL END) AS caught
	FROM  deliveries
    WHERE wicket_type NOT IN ( 'run out','retired out','obstructing the field')
	GROUP BY season,bowler
    )

SELECT season,
    bowler, 
    total_wicket,
    stumped_wicket,
    lbw_wicket,
    caught_and_bowled_wicket,
    hit_wicket,
    caught,
    bowled_wicket
FROM(SELECT *, 
        ROW_NUMBER() OVER(partition by season ORDER BY total_wicket DESC) AS rnk
    FROM cte) A
WHERE rnk=1
"""
).show()

+-------+--------------+------------+--------------+----------+------------------------+----------+------+-------------+
| season|        bowler|total_wicket|stumped_wicket|lbw_wicket|caught_and_bowled_wicket|hit_wicket|caught|bowled_wicket|
+-------+--------------+------------+--------------+----------+------------------------+----------+------+-------------+
|2007/08| Sohail Tanvir|          22|             0|         3|                       0|         0|    11|            8|
|   2009|      RP Singh|          23|             0|         0|                       1|         0|    17|            5|
|2009/10|       PP Ojha|          21|             4|         1|                       0|         0|    12|            4|
|   2011|    SL Malinga|          28|             0|         0|                       2|         0|    10|           16|
|   2012|      M Morkel|          25|             0|         1|                       0|         0|    16|            8|
|   2013|      DJ Bravo|        

 ***8. Getting the frequency of most man of the match awards***

In [11]:
spark.sql('''
SELECT player_of_match,
    COUNT(player_of_match) AS total_mom
FROM matches
GROUP BY player_of_match
ORDER BY total_mom DESC
LIMIT 1
''').show(truncate=False)

+---------------+---------+
|player_of_match|total_mom|
+---------------+---------+
|AB de Villiers |25       |
+---------------+---------+



***9. Finding number of toss wins w.r.t each team***

In [12]:
spark.sql('''
SELECT team_1,
    team_2,
    SUM(total_toss_win) AS total_win_by_team1
FROM(
    SELECT team1 AS team_1,
        team2 AS team_2,
        SUM( CASE WHEN team1=toss_winner THEN 1 ELSE 0 END) AS total_toss_win
    FROM matches
    GROUP BY team1,team2

    UNION ALL
    
    SELECT team2 AS team_1,
        team1 AS team_2, 
        SUM( CASE WHEN team2=toss_winner THEN 1 ELSE 0 END) AS total_toss_win
    FROM matches
    GROUP BY team2,team1 ) 
GROUP BY team_1,team_2
ORDER BY team_1,team_2
''').show()

+-------------------+--------------------+------------------+
|             team_1|              team_2|total_win_by_team1|
+-------------------+--------------------+------------------+
|Chennai Super Kings|     Deccan Chargers|                 5|
|Chennai Super Kings|      Delhi Capitals|                 6|
|Chennai Super Kings|    Delhi Daredevils|                11|
|Chennai Super Kings|      Gujarat Titans|                 2|
|Chennai Super Kings|     Kings XI Punjab|                15|
|Chennai Super Kings|Kochi Tuskers Kerala|                 1|
|Chennai Super Kings|Kolkata Knight Ri...|                13|
|Chennai Super Kings|Lucknow Super Giants|                 1|
|Chennai Super Kings|      Mumbai Indians|                18|
|Chennai Super Kings|       Pune Warriors|                 2|
|Chennai Super Kings|        Punjab Kings|                 4|
|Chennai Super Kings|    Rajasthan Royals|                11|
|Chennai Super Kings|Royal Challengers...|                20|
|Chennai

***10. Which team most win after batting first has won the match***

In [19]:
spark.sql('''
SELECT winner, 
    SUM(CASE WHEN (toss_winner=winner AND toss_decision='bat') OR 
        (toss_winner!=winner AND toss_decision='field') THEN 1 ELSE 0 END) AS total_win_after_batting_first
FROM matches
GROUP BY winner
''').show(truncate=False)

+---------------------------+-----------------------------+
|winner                     |total_win_after_batting_first|
+---------------------------+-----------------------------+
|Sunrisers Hyderabad        |38                           |
|Lucknow Super Giants       |12                           |
|Chennai Super Kings        |66                           |
|Gujarat Titans             |9                            |
|NULL                       |0                            |
|Rising Pune Supergiant     |5                            |
|Deccan Chargers            |18                           |
|Kochi Tuskers Kerala       |2                            |
|Rajasthan Royals           |41                           |
|Gujarat Lions              |1                            |
|Royal Challengers Bangalore|53                           |
|Kolkata Knight Riders      |46                           |
|Rising Pune Supergiants    |0                            |
|Kings XI Punjab            |39         

***11. Find out how many times a team has won the match after winning the toss***

In [28]:
spark.sql('''
SELECT team,
    SUM(total_win) AS total_wins
FROM	
    (SELECT team1 AS team,
        SUM( CASE WHEN toss_winner=winner and winner =team1 THEN 1 ELSE 0 END ) as total_win
	FROM matches
	GROUP BY team1
 
	UNION ALL
 
	SELECT team2,
         SUM( CASE WHEN toss_winner=winner and winner =team2 THEN 1 ELSE 0 END ) 
	FROM matches
	GROUP BY team2)
GROUP BY team
ORDER BY total_wins DESC
'''    
).show(truncate=False)

+---------------------------+----------+
|team                       |total_wins|
+---------------------------+----------+
|Mumbai Indians             |75        |
|Chennai Super Kings        |74        |
|Kolkata Knight Riders      |65        |
|Royal Challengers Bangalore|56        |
|Rajasthan Royals           |52        |
|Sunrisers Hyderabad        |36        |
|Kings XI Punjab            |35        |
|Delhi Daredevils           |35        |
|Delhi Capitals             |22        |
|Deccan Chargers            |19        |
|Gujarat Titans             |13        |
|Gujarat Lions              |10        |
|Lucknow Super Giants       |5         |
|Rising Pune Supergiant     |5         |
|Punjab Kings               |5         |
|Kochi Tuskers Kerala       |4         |
|Rising Pune Supergiants    |3         |
|Pune Warriors              |3         |
+---------------------------+----------+



***12. Which city hosted the most number of matches?***

In [30]:
spark.sql('''
SELECT city,
    COUNT(*) AS most_number_of_matches_hosted
FROM matches
GROUP BY city
ORDER BY most_number_of_matches_hosted DESC
''').show()

+-------------+-----------------------------+
|         city|most_number_of_matches_hosted|
+-------------+-----------------------------+
|       Mumbai|                          166|
|      Kolkata|                           86|
|        Delhi|                           85|
|      Chennai|                           76|
|    Hyderabad|                           71|
|    Bangalore|                           65|
|   Chandigarh|                           61|
|       Jaipur|                           52|
|         NULL|                           51|
|         Pune|                           51|
|    Abu Dhabi|                           37|
|    Ahmedabad|                           28|
|    Bengaluru|                           22|
|       Durban|                           15|
|        Dubai|                           13|
|Visakhapatnam|                           13|
|    Centurion|                           12|
|   Dharamsala|                           11|
|      Sharjah|                   

***13.which team won most number of toss***

In [46]:
spark.sql('''
SELECT team,
    SUM(total) AS total_won_toss,
    SUM(total_m) AS total_match_played,
    ROUND(100*SUM(total)/SUM(total_m),2) AS toss_winning_percentage
FROM(
    SELECT team1 AS team,
        COUNT(DISTINCT id) AS  total_m,
        SUM(IF(team1 = toss_winner,1,0)) AS total
    FROM matches
    GROUP BY team1
    
    UNION ALL
    
    SELECT team2,
        COUNT(DISTINCT id),
        SUM(IF(team2 = toss_winner,1,0)) 
    FROM matches
    GROUP BY team2
    ) 
GROUP BY team
ORDER BY total_won_toss DESC,toss_winning_percentage DESC
''').show(truncate=False)

+---------------------------+--------------+------------------+-----------------------+
|team                       |total_won_toss|total_match_played|toss_winning_percentage|
+---------------------------+--------------+------------------+-----------------------+
|Mumbai Indians             |133           |247               |53.85                  |
|Chennai Super Kings        |119           |224               |53.13                  |
|Kolkata Knight Riders      |119           |237               |50.21                  |
|Royal Challengers Bangalore|113           |240               |47.08                  |
|Rajasthan Royals           |109           |206               |52.91                  |
|Kings XI Punjab            |85            |190               |44.74                  |
|Sunrisers Hyderabad        |81            |166               |48.8                   |
|Delhi Daredevils           |80            |161               |49.69                  |
|Deccan Chargers            |43 

In [36]:
spark.sql('''
SELECT team1,
    SUM(CASE WHEN (team1 = toss_winner) THEN 1 ELSE 0 END) AS total_num1
FROM matches
GROUP BY team1
''').show(truncate=False)

+---------------------------+----------+
|team1                      |total_num1|
+---------------------------+----------+
|Sunrisers Hyderabad        |29        |
|Lucknow Super Giants       |2         |
|Chennai Super Kings        |52        |
|Gujarat Titans             |6         |
|Rising Pune Supergiant     |5         |
|Deccan Chargers            |22        |
|Kochi Tuskers Kerala       |4         |
|Rajasthan Royals           |38        |
|Gujarat Lions              |7         |
|Royal Challengers Bangalore|47        |
|Kolkata Knight Riders      |43        |
|Rising Pune Supergiants    |2         |
|Kings XI Punjab            |34        |
|Punjab Kings               |3         |
|Pune Warriors              |10        |
|Delhi Daredevils           |39        |
|Delhi Capitals             |10        |
|Mumbai Indians             |53        |
+---------------------------+----------+



***14.Number of matches played in each season and no. of balls?***

In [56]:
spark.sql('''
SELECT season,
    COUNT(DISTINCT match_id) AS total_match_played,
    COUNT(ball) AS no_of_balls
FROM deliveries
GROUP BY season
ORDER BY Season 
''').show()

+-------+------------------+-----------+
| season|total_match_played|no_of_balls|
+-------+------------------+-----------+
|2007/08|                58|      13489|
|   2009|                57|      13606|
|2009/10|                60|      14498|
|   2011|                73|      17013|
|   2012|                74|      17767|
|   2013|                76|      18177|
|   2014|                60|      14300|
|   2015|                59|      13652|
|   2016|                60|      14096|
|   2017|                59|      13862|
|   2018|                60|      14286|
|   2019|                60|      14312|
|2020/21|                60|      14559|
|   2021|                60|      14425|
|   2022|                74|      17912|
|   2023|                74|      17863|
+-------+------------------+-----------+



***15. Highest & lowest run scored by a batsman?***

In [64]:
spark.sql('''
WITH batsman_runs AS (
    SELECT season, 
        match_id,
        striker,
        SUM(runs_off_bat) AS total_batsman_run
    FROM deliveries
    GROUP BY 1,2,3
)
SELECT season,
    striker,
    MAX(total_batsman_run) AS Highest_runs,
    MIN(total_batsman_run) AS  Lowest_runs
FROM batsman_runs
GROUP BY 1,2
ORDER BY season DESC,Highest_runs DESC ,Lowest_runs
''').show()

+------+---------------+------------+-----------+
|season|        striker|Highest_runs|Lowest_runs|
+------+---------------+------------+-----------+
|  2023|   Shubman Gill|         129|          0|
|  2023|    YBK Jaiswal|         124|          0|
|  2023|        VR Iyer|         104|          0|
|  2023|      H Klaasen|         104|         16|
|  2023|       SA Yadav|         103|          0|
|  2023| P Simran Singh|         103|          0|
|  2023|        V Kohli|         101|          0|
|  2023|       HC Brook|         100|          0|
|  2023|        C Green|         100|          1|
|  2023|       S Dhawan|          99|          0|
|  2023|B Sai Sudharsan|          96|         19|
|  2023|     JC Buttler|          95|          0|
|  2023| LS Livingstone|          94|          2|
|  2023|      DP Conway|          92|          0|
|  2023|     RD Gaikwad|          92|          3|
|  2023|     MP Stoinis|          89|          0|
|  2023|        PD Salt|          87|          0|


***16. To find the player who has scored the most centuries in each season of the IPL, and in case of a tie, print the highest score of the batsman***

In [20]:
spark.sql('''
WITH batsman_runs AS (
    SELECT season, 
        match_id,
        striker,
        SUM(runs_off_bat) AS total_batsman_run
    FROM deliveries
    GROUP BY 1,2,3
)
SELECT season,striker,most_century,highest_score
FROM(
    SELECT season,
        striker,
        most_century,
        highest_score,
        DENSE_RANK() OVER(PARTITION BY season ORDER BY most_century DESC,highest_score DESC) AS rnk
    FROM(
        SELECT season, 
                striker,
                COUNT(total_batsman_run) AS most_century,
                MAX(total_batsman_run) AS highest_score
        FROM batsman_runs
        WHERE total_batsman_run>=100
        GROUP BY 1,2
        ORDER BY season DESC
        ) A
    ) B
WHERE rnk=1
ORDER BY season DESC

''').show()

+-------+--------------+------------+-------------+
| season|       striker|most_century|highest_score|
+-------+--------------+------------+-------------+
|   2023|  Shubman Gill|           3|          129|
|   2022|    JC Buttler|           4|          116|
|   2021|    JC Buttler|           1|          124|
|2020/21|      S Dhawan|           2|          106|
|   2019|   JM Bairstow|           1|          114|
|   2018|     SR Watson|           2|          117|
|   2017|       HM Amla|           2|          104|
|   2016|       V Kohli|           4|          113|
|   2015|AB de Villiers|           1|          133|
|   2014|      V Sehwag|           1|          122|
|   2013|      CH Gayle|           1|          175|
|   2012|      CH Gayle|           1|          128|
|   2011|      CH Gayle|           2|          107|
|2009/10|       M Vijay|           1|          127|
|   2009|     MK Pandey|           1|          114|
|2007/08|   BB McCullum|           1|          158|
+-------+---

***16. find the player who has highest score along with batsman name and total no. of century in each season of the IPL***

In [26]:
spark.sql('''
WITH batsman_runs AS (
    SELECT season, 
        match_id,
        striker,
        SUM(runs_off_bat) AS total_batsman_run
    FROM deliveries
    GROUP BY 1,2,3
)
SELECT season,
    total_century,
    highest_score,
    striker
   
FROM(
    SELECT *, 
            COUNT(total_batsman_run) OVER(PARTITION BY season ) AS total_century,
            MAX(total_batsman_run) OVER(PARTITION BY season ) AS highest_score,
            ROW_NUMBER() OVER(PARTITION BY season ORDER BY total_batsman_run DESC) as rnk
    FROM batsman_runs
    WHERE total_batsman_run>=100

    ) A
WHERE rnk=1
ORDER BY season DESC

''').show()

+-------+-------------+-------------+--------------+
| season|total_century|highest_score|       striker|
+-------+-------------+-------------+--------------+
|   2023|           12|          129|  Shubman Gill|
|   2022|            8|          140|     Q de Kock|
|   2021|            4|          124|    JC Buttler|
|2020/21|            5|          132|      KL Rahul|
|   2019|            6|          114|   JM Bairstow|
|   2018|            5|          128|       RR Pant|
|   2017|            5|          126|     DA Warner|
|   2016|            7|          129|AB de Villiers|
|   2015|            4|          133|AB de Villiers|
|   2014|            3|          122|      V Sehwag|
|   2013|            4|          175|      CH Gayle|
|   2012|            6|          128|      CH Gayle|
|   2011|            6|          120|   PC Valthaty|
|2009/10|            4|          127|       M Vijay|
|   2009|            2|          114|     MK Pandey|
|2007/08|            6|          158|   BB McC

***17. highest & lowest score in each season?***

In [101]:
spark.sql('''
WITH inning_scores AS (
    SELECT season,
        match_id,
        innings,
        SUM(runs_off_bat+extras) AS inning_score
    FROM deliveries
    WHERE (innings=2 OR innings=1) AND match_id <> 829813 AND
        match_id IN(SELECT DISTINCT id
                        FROM matches
                        WHERE result <> 'D/L')
    GROUP BY 1,2,3
)
SELECT season,
    MAX(inning_score) AS Highest_score,
    MIN(inning_score) AS Lowest_score
FROM inning_scores
GROUP BY 1
ORDER BY season,Highest_score DESC,Lowest_score
''').show()

+-------+-------------+------------+
| season|Highest_score|Lowest_score|
+-------+-------------+------------+
|2007/08|          240|          67|
|   2009|          211|          58|
|2009/10|          246|          82|
|   2011|          232|          56|
|   2012|          222|          92|
|   2013|          263|          79|
|   2014|          231|          70|
|   2015|          235|          84|
|   2016|          248|          92|
|   2017|          230|          49|
|   2018|          245|          87|
|   2019|          232|          41|
|2020/21|          228|          84|
|   2021|          235|          85|
|   2022|          222|          68|
|   2023|          257|          59|
+-------+-------------+------------+



***18. Which bowler has best Bowling Average in each season and minimum 10 wickets has taken***

In [51]:
spark.sql('''
WITH bowling_avg AS(
    SELECT season,bowler,
        SUM(runs_off_bat+extras) AS run_conceded,
        COUNT(CASE WHEN wicket_type NOT IN( 'run out','retired out','obstructing the field') THEN player_dismissed ELSE NULL END) AS wicket_taken,
        ROUND(run_conceded/wicket_taken,2) AS `Bowling Average`
    FROM deliveries 
    WHERE penalty IS NULL 
    GROUP BY 1,2
    HAVING wicket_taken>=10
    ORDER BY season DESC,`Bowling Average`
    )
SELECT season,bowler,run_conceded,wicket_taken,`Bowling Average`
FROM(
    SELECT *,
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY `Bowling Average`) AS rnk 
    FROM bowling_avg
    ) A
WHERE rnk=1
''').show(truncate=False)

+-------+---------------+------------+------------+---------------+
|season |bowler         |run_conceded|wicket_taken|Bowling Average|
+-------+---------------+------------+------------+---------------+
|2007/08|Sohail Tanvir  |275         |22          |12.5           |
|2009   |RG Sharma      |163         |11          |14.82          |
|2009/10|RJ Harris      |239         |14          |17.07          |
|2011   |SL Malinga     |393         |28          |14.04          |
|2012   |SP Narine      |332         |24          |13.83          |
|2013   |JP Faulkner    |436         |28          |15.57          |
|2014   |Karanveer Singh|164         |11          |14.91          |
|2015   |MC Henriques   |164         |11          |14.91          |
|2016   |A Zampa        |117         |12          |9.75           |
|2017   |P Negi         |202         |17          |11.88          |
|2018   |L Ngidi        |162         |11          |14.73          |
|2019   |K Rabada       |382         |26        

***19. Which bowler has best Strike Rate in each season and minimum 10 wickets has taken***

In [53]:
spark.sql('''
WITH strike_rate AS(
    SELECT season,bowler,
        COUNT(ball) AS ball_bowled,
        COUNT(CASE WHEN wicket_type NOT IN( 'run out','retired out','obstructing the field') THEN player_dismissed ELSE NULL END) AS wicket_taken,
        ROUND(ball_bowled/wicket_taken,2) AS `Strike Rate`
    FROM deliveries 
    WHERE penalty IS NULL 
    GROUP BY 1,2
    HAVING wicket_taken>=10
  
    )
SELECT season,bowler,ball_bowled,wicket_taken,`Strike Rate`
FROM(
    SELECT *,
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY `Strike Rate`) AS rnk 
    FROM strike_rate
    ) A
WHERE rnk=1
''').show(truncate=False)

+-------+---------------+-----------+------------+-----------+
|season |bowler         |ball_bowled|wicket_taken|Strike Rate|
+-------+---------------+-----------+------------+-----------+
|2007/08|A Mishra       |123        |11          |11.18      |
|2009   |RG Sharma      |140        |11          |12.73      |
|2009/10|RJ Harris      |195        |14          |13.93      |
|2011   |S Aravind      |288        |21          |13.71      |
|2012   |SP Narine      |357        |24          |14.88      |
|2013   |DJ Bravo       |392        |32          |12.25      |
|2014   |Karanveer Singh|126        |11          |11.45      |
|2015   |YS Chahal      |283        |23          |12.3       |
|2016   |A Zampa        |103        |12          |8.58       |
|2017   |AJ Tye         |128        |12          |10.67      |
|2018   |AJ Tye         |353        |24          |14.71      |
|2019   |K Rabada       |298        |26          |11.46      |
|2020/21|JO Holder      |176        |14          |12.57

***20. Which bowler has best Economy Rate in each season and minimum 20 OVERS has thrown***

In [75]:
spark.sql('''
WITH economy_rate AS(
        SELECT season,
            match_id,
            bowler,
            COUNT(DISTINCT over) AS over_bowled,
            SUM(runs_off_bat+extras) AS run_conceded
            
        FROM(
            SELECT *,
                CAST(ball AS int) AS over 
            FROM deliveries
            ) 
        WHERE penalty IS NULL 
        GROUP BY 1,2,3
        ),
    total_bowled_overs AS(
        SELECT season,
            bowler,
            SUM(over_bowled) AS total_over_bowled,
            SUM(run_conceded) AS total_runs,
            ROUND(total_runs/total_over_bowled,2) AS `Economy Rate`
        FROM economy_rate
        GROUP BY 1,2
        HAVING total_over_bowled>=20
        )
    

SELECT season,bowler,total_over_bowled,total_runs,`Economy Rate`
FROM(
    SELECT *,
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY `Economy Rate`) AS rnk 
    FROM total_bowled_overs
    ) A
WHERE rnk=1
''').show(truncate=False)

+-------+-----------------+-----------------+----------+------------+
|season |bowler           |total_over_bowled|total_runs|Economy Rate|
+-------+-----------------+-----------------+----------+------------+
|2007/08|Sohail Tanvir    |42               |275       |6.55        |
|2009   |M Muralitharan   |50               |267       |5.34        |
|2009/10|R Ashwin         |48               |308       |6.42        |
|2011   |R Sharma         |50               |278       |5.56        |
|2012   |SP Narine        |60               |332       |5.53        |
|2013   |SP Narine        |64               |353       |5.52        |
|2014   |AR Patel         |66               |424       |6.42        |
|2015   |R Ashwin         |39               |229       |5.87        |
|2016   |Mustafizur Rahman|61               |427       |7.0         |
|2017   |P Negi           |33               |202       |6.12        |
|2018   |IS Sodhi         |23               |137       |5.96        |
|2019   |RA Jadeja  

***21.Which Batsman has hit maximum sixes and fours?***


In [11]:
spark.sql('''
SELECT striker,
    SUM(CASE WHEN runs_off_bat=6 THEN 1 ELSE 0 END) as sixes
FROM deliveries
GROUP BY 1
ORDER BY sixes DESC
LIMIT 1
''').show(truncate=False)

+--------+-----+
|striker |sixes|
+--------+-----+
|CH Gayle|359  |
+--------+-----+



In [12]:
spark.sql('''
SELECT striker,
    SUM(CASE WHEN runs_off_bat=4 THEN 1 ELSE 0 END) as Fours
FROM deliveries
GROUP BY 1
ORDER BY fours DESC
LIMIT 1
''').show(truncate=False)

+--------+-----+
|striker |Fours|
+--------+-----+
|S Dhawan|750  |
+--------+-----+



***22. Calculate the average runs scored by each team in each season.***

In [57]:
spark.sql('''
WITH team_scores AS(
    SELECT season,
        match_id,
        batting_team AS team,
        SUM(runs_off_bat+extras) as run_scored
    FROM deliveries
    GROUP BY 1,2,3
    ORDER BY season DESC,match_id desc
)
SELECT season,team,
    ROUND(AVG(run_scored),2) AS average_runs
FROM team_scores
GROUP BY season,team
ORDER BY season DESC,average_runs DESC
''').show(truncate=False)

+------+---------------------------+------------+
|season|team                       |average_runs|
+------+---------------------------+------------+
|2023  |Mumbai Indians             |184.06      |
|2023  |Punjab Kings               |182.57      |
|2023  |Chennai Super Kings        |180.8       |
|2023  |Gujarat Titans             |179.65      |
|2023  |Royal Challengers Bangalore|178.71      |
|2023  |Kolkata Knight Riders      |175.93      |
|2023  |Rajasthan Royals           |172.79      |
|2023  |Sunrisers Hyderabad        |169.71      |
|2023  |Lucknow Super Giants       |165.27      |
|2023  |Delhi Capitals             |155.86      |
|2022  |Rajasthan Royals           |173.12      |
|2022  |Lucknow Super Giants       |169.87      |
|2022  |Punjab Kings               |167.36      |
|2022  |Delhi Capitals             |167.21      |
|2022  |Gujarat Titans             |166.44      |
|2022  |Royal Challengers Bangalore|164.5       |
|2022  |Chennai Super Kings        |163.43      |


***23. Calculate the total number of runs scored by each team in the powerplay (first 6 overs) in each season.***

In [61]:
spark.sql('''
WITH PowerPlay AS(
     SELECT season,
        match_id,
        batting_team AS team,
        SUM(runs_off_bat+extras) AS powerplay_runs
        
    FROM(
        SELECT *,
            CAST(ball AS int) AS over 
        FROM deliveries
        ) 
    WHERE penalty IS NULL AND over<=5
    GROUP BY 1,2,3
)  
SELECT season,team,SUM(powerplay_runs) AS total_runs_in_powerplay
FROM PowerPlay
GROUP BY season,team
ORDER BY season DESC,total_runs_in_powerplay DESC
''').show(truncate=False)

+------+---------------------------+-----------------------+
|season|team                       |total_runs_in_powerplay|
+------+---------------------------+-----------------------+
|2023  |Gujarat Titans             |888                    |
|2023  |Mumbai Indians             |887                    |
|2023  |Chennai Super Kings        |843                    |
|2023  |Rajasthan Royals           |790                    |
|2023  |Royal Challengers Bangalore|777                    |
|2023  |Punjab Kings               |730                    |
|2023  |Lucknow Super Giants       |715                    |
|2023  |Kolkata Knight Riders      |700                    |
|2023  |Delhi Capitals             |692                    |
|2023  |Sunrisers Hyderabad        |665                    |
|2022  |Rajasthan Royals           |854                    |
|2022  |Punjab Kings               |773                    |
|2022  |Delhi Capitals             |758                    |
|2022  |Gujarat Titans  

***23. Which team has most of runs scored in the powerplay (first 6 overs) in each season.***

In [13]:
spark.sql('''
WITH PowerPlay AS(
     SELECT season,
        match_id,
        batting_team AS team,
        SUM(runs_off_bat+extras) AS powerplay_runs
        
    FROM(
        SELECT *,
            CAST(ball AS int) AS over 
        FROM deliveries
        ) 
    WHERE penalty IS NULL AND over<=5
    GROUP BY 1,2,3
)  
SELECT season,team,powerplay_runs
FROM(
    SELECT season,
        team,
        powerplay_runs,
        ROW_NUMBER() OVER(PARTITION BY season ORDER BY powerplay_runs DESC) AS rnk
    FROM PowerPlay
    ORDER BY  season DESC
    )
WHERE rnk=1
''').show(truncate=False)

+-------+---------------------------+--------------+
|season |team                       |powerplay_runs|
+-------+---------------------------+--------------+
|2023   |Rajasthan Royals           |85            |
|2022   |Punjab Kings               |83            |
|2021   |Mumbai Indians             |83            |
|2020/21|Sunrisers Hyderabad        |77            |
|2019   |Sunrisers Hyderabad        |77            |
|2018   |Mumbai Indians             |84            |
|2017   |Kolkata Knight Riders      |105           |
|2016   |Gujarat Lions              |72            |
|2015   |Chennai Super Kings        |90            |
|2014   |Chennai Super Kings        |100           |
|2013   |Royal Challengers Bangalore|76            |
|2012   |Kolkata Knight Riders      |68            |
|2011   |Kochi Tuskers Kerala       |87            |
|2009/10|Kings XI Punjab            |76            |
|2009   |Deccan Chargers            |84            |
|2007/08|Mumbai Indians             |78       

***24. Which team has 100+ runs scored in the powerplay (first 6 overs) in each season.***

In [68]:
spark.sql('''
WITH PowerPlay AS(
     SELECT season,
        match_id,
        batting_team AS team,
        SUM(runs_off_bat+extras) AS powerplay_runs
        
    FROM(
        SELECT *,
            CAST(ball AS int) AS over 
        FROM deliveries
        ) 
    WHERE penalty IS NULL AND over<=5
    GROUP BY 1,2,3
)  
SELECT season,team,powerplay_runs
FROM(
    SELECT season,
        team,
        powerplay_runs
    FROM PowerPlay
    WHERE powerplay_runs>=100
    ORDER BY  season DESC
    )
''').show(truncate=False)

+------+---------------------+--------------+
|season|team                 |powerplay_runs|
+------+---------------------+--------------+
|2017  |Kolkata Knight Riders|105           |
|2014  |Chennai Super Kings  |100           |
+------+---------------------+--------------+



***25. Which team has 100+ runs scored in the death overs (first 6 overs) in each season.***

In [76]:
spark.sql('''
WITH DeathOver AS(
     SELECT season,
        match_id,
        batting_team AS team,
        SUM(runs_off_bat+extras) AS death_overs_runs
        
    FROM(
        SELECT *,
            CAST(ball AS int) AS over 
        FROM deliveries
        ) 
    WHERE penalty IS NULL AND over BETWEEN 15 AND 20
    GROUP BY 1,2,3
)  
SELECT season,team,death_overs_runs
FROM(
    SELECT season,
        team,
        death_overs_runs
    FROM DeathOver
    WHERE death_overs_runs>=100
    ORDER BY  season DESC
    )
''').show(truncate=False)

+------+---------------------------+----------------+
|season|team                       |death_overs_runs|
+------+---------------------------+----------------+
|2016  |Royal Challengers Bangalore|112             |
+------+---------------------------+----------------+



***26. Find the team that has won the most matches by a margin of more than 50 runs.***

In [79]:
spark.sql('''
SELECT winner,
    COUNT(id) AS most_won_matches
FROM matches
WHERE win_by_runs>=50
GROUP BY winner
ORDER BY most_won_matches DESC
''').show(truncate=False)

+---------------------------+----------------+
|winner                     |most_won_matches|
+---------------------------+----------------+
|Chennai Super Kings        |16              |
|Mumbai Indians             |13              |
|Royal Challengers Bangalore|11              |
|Rajasthan Royals           |10              |
|Kolkata Knight Riders      |9               |
|Kings XI Punjab            |6               |
|Sunrisers Hyderabad        |4               |
|Gujarat Titans             |4               |
|Delhi Daredevils           |4               |
|Lucknow Super Giants       |3               |
|Deccan Chargers            |3               |
|Punjab Kings               |2               |
|Rising Pune Supergiant     |1               |
|Delhi Capitals             |1               |
+---------------------------+----------------+



***27. Calculate the total number of sixes hit by each team in each season.***

In [82]:
spark.sql('''
SELECT season,
    batting_team,
    SUM(CASE WHEN runs_off_bat=6 THEN 1 ELSE 0 END) as sixes
FROM deliveries
GROUP BY 1,2
ORDER BY season DESC,sixes DESC
''').show(truncate=False)

+------+---------------------------+-----+
|season|batting_team               |sixes|
+------+---------------------------+-----+
|2023  |Mumbai Indians             |140  |
|2023  |Chennai Super Kings        |133  |
|2023  |Kolkata Knight Riders      |125  |
|2023  |Gujarat Titans             |124  |
|2023  |Punjab Kings               |117  |
|2023  |Lucknow Super Giants       |115  |
|2023  |Rajasthan Royals           |112  |
|2023  |Royal Challengers Bangalore|107  |
|2023  |Sunrisers Hyderabad        |84   |
|2023  |Delhi Capitals             |67   |
|2022  |Rajasthan Royals           |137  |
|2022  |Lucknow Super Giants       |115  |
|2022  |Kolkata Knight Riders      |113  |
|2022  |Punjab Kings               |110  |
|2022  |Delhi Capitals             |106  |
|2022  |Chennai Super Kings        |103  |
|2022  |Royal Challengers Bangalore|102  |
|2022  |Mumbai Indians             |100  |
|2022  |Sunrisers Hyderabad        |97   |
|2022  |Gujarat Titans             |79   |
+------+---

***28. Identify the venue that has hosted the most number of matches, and the average runs scored at that venue.***

In [50]:
spark.sql('''
WITH MostMatches AS (
        SELECT venue,
            SUM(most_number_of_matches) AS most_matches
        FROM(
            SELECT venue,
                match_id,
                COUNT(DISTINCT match_id) AS most_number_of_matches
            FROM deliveries
            GROUP BY venue,match_id
            ORDER BY most_number_of_matches DESC
            )
        GROUP BY venue
        ORDER BY most_matches DESC
            ),
    AverageRuns AS (
        SELECT venue,ROUND(AVG(total_runs),2) AS average_runs
        FROM(
            SELECT venue,
                match_id,
                innings,
                SUM(runs_off_bat + extras) AS total_runs
            FROM deliveries
            WHERE innings =1 OR innings=2
            GROUP BY venue,match_id,innings
            ORDER BY venue,match_id,innings
            )
        GROUP BY venue
        ORDER BY average_runs DESC
        )
SELECT m.venue,m.most_matches,a.average_runs
FROM MostMatches m
JOIN AverageRuns a ON m.venue=a.venue
ORDER BY most_matches DESC
''').show(truncate=False)

+------------------------------------------+------------+------------+
|venue                                     |most_matches|average_runs|
+------------------------------------------+------------+------------+
|Eden Gardens                              |77          |153.62      |
|Wankhede Stadium                          |73          |160.21      |
|M Chinnaswamy Stadium                     |65          |156.88      |
|Feroz Shah Kotla                          |60          |154.8       |
|Rajiv Gandhi International Stadium, Uppal |49          |151.56      |
|MA Chidambaram Stadium, Chepauk           |48          |158.94      |
|Sawai Mansingh Stadium                    |47          |151.74      |
|Dubai International Cricket Stadium       |46          |156.42      |
|Wankhede Stadium, Mumbai                  |38          |171.55      |
|Punjab Cricket Association Stadium, Mohali|35          |156.96      |
|Sheikh Zayed Stadium                      |29          |152.24      |
|Sharj

***29. Top 10 plyaers has won the "Player of the Match" awards in IPL history***

In [58]:
spark.sql('''
SELECT player_of_match,
    COUNT(DISTINCT id) AS Most_POM
FROM matches
GROUP BY player_of_match
ORDER BY Most_POM DESC
LIMIT 10
''').show(truncate=False)

+---------------+--------+
|player_of_match|Most_POM|
+---------------+--------+
|AB de Villiers |25      |
|CH Gayle       |22      |
|RG Sharma      |19      |
|DA Warner      |18      |
|MS Dhoni       |17      |
|YK Pathan      |16      |
|SR Watson      |16      |
|V Kohli        |16      |
|KA Pollard     |14      |
|SK Raina       |14      |
+---------------+--------+



***30. Which team has the best record in super overs?***

In [62]:
spark.sql('''
SELECT *
FROM matches
WHERE id IN(
    SELECT DISTINCT match_id
    FROM deliveries
    WHERE innings>2
    )
''').show(truncate=False)

+-------+-------+---------+----------+---------------------------+---------------------------+---------------------------+-------------+------+----------+------+-----------+--------------+---------------+-----------------------------------------+------------+---------------------+--------------+
|id     |season |city     |date      |team1                      |team2                      |toss_winner                |toss_decision|result|dl_applied|winner|win_by_runs|win_by_wickets|player_of_match|venue                                    |umpire1     |umpire2              |umpire3       |
+-------+-------+---------+----------+---------------------------+---------------------------+---------------------------+-------------+------+----------+------+-----------+--------------+---------------+-----------------------------------------+------------+---------------------+--------------+
|1254077|2021   |Chennai  |2021/04/25|Delhi Capitals             |Sunrisers Hyderabad        |Delhi Capitals 

***31. Which team has the most number of hat-tricks in IPL history?***

In [18]:
spark.sql('''
SELECT bowler,COUNT(player_dismissed) AS total_hat_ricks
FROM(
    SELECT bowler,player_dismissed,
        LEAD(player_dismissed) OVER(PARTITION BY match_id,innings,bowler ORDER BY match_id ) AS next_ball,
        LEAD(player_dismissed,2) OVER(PARTITION BY match_id,innings,bowler ORDER BY match_id ) AS next_to_next_ball
    
    FROM deliveries 
    )
WHERE player_dismissed IS NOT NULL AND next_ball IS NOT NULL AND next_to_next_ball IS NOT NULL
GROUP BY bowler 
ORDER BY total_hat_ricks DESC
''').show(truncate=False)

+-----------------+---------------+
|bowler           |total_hat_ricks|
+-----------------+---------------+
|A Mishra         |4              |
|Yuvraj Singh     |2              |
|MM Sharma        |2              |
|AR Patel         |1              |
|HV Patel         |1              |
|RG Sharma        |1              |
|SK Trivedi       |1              |
|B Kumar          |1              |
|JD Unadkat       |1              |
|A Chandila       |1              |
|SM Curran        |1              |
|SP Narine        |1              |
|HH Pandya        |1              |
|Rashid Khan      |1              |
|M Ntini          |1              |
|M Prasidh Krishna|1              |
|S Badree         |1              |
|AJ Tye           |1              |
|R Vinay Kumar    |1              |
|YS Chahal        |1              |
+-----------------+---------------+
only showing top 20 rows



***32. Which player has the most number of ducks in IPL history?***

In [20]:
spark.sql('''
WITH batsman_runs AS (
    SELECT  match_id,
        striker,
        SUM(runs_off_bat) AS total_batsman_run
    FROM deliveries
    GROUP BY 1,2
)
SELECT striker,COUNT(*) AS most_number_of_ducks
FROM batsman_runs
WHERE total_batsman_run = 0
GROUP BY striker
ORDER BY most_number_of_ducks DESC
''').show(truncate=False)

+---------------+--------------------+
|striker        |most_number_of_ducks|
+---------------+--------------------+
|KD Karthik     |18                  |
|SP Narine      |16                  |
|RG Sharma      |15                  |
|Rashid Khan    |15                  |
|GJ Maxwell     |14                  |
|MK Pandey      |14                  |
|Mandeep Singh  |14                  |
|R Ashwin       |13                  |
|AM Rahane      |13                  |
|AT Rayudu      |13                  |
|PA Patel       |13                  |
|PP Chawla      |13                  |
|S Dhawan       |12                  |
|Harbhajan Singh|12                  |
|DA Warner      |11                  |
|G Gambhir      |11                  |
|CH Morris      |10                  |
|YK Pathan      |10                  |
|UT Yadav       |10                  |
|V Kohli        |10                  |
+---------------+--------------------+
only showing top 20 rows



***(NA)33. Which team has the most number of century partnerships in IPL history?***

In [22]:
spark.sql('''
SELECT *
FROM deliveries
''').show(truncate=False)

+--------+------+----------+--------------------------------+-------+----+--------------+-------------------+------------+------------+------------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+------------+
|match_id|season|start_date|venue                           |innings|ball|batting_team  |bowling_team       |striker     |non_striker |bowler      |runs_off_bat|extras|wides|noballs|byes|legbyes|penalty|wicket_type|player_dismissed|other_wicket_type|other_player_dismissed|cricsheet_id|
+--------+------+----------+--------------------------------+-------+----+--------------+-------------------+------------+------------+------------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+------------+
|1370353 |2023  |2023-05-29|Narendra Modi Stadium, Ahmedabad|1      |0.1 |Gujarat Titans|Chennai Super Kings|WP Saha     |Shubman Gill|DL C

***34. Which batsman has the highest strike rate in the last five overs of an innings in IPL history?***

In [68]:
spark.sql("""
SELECT striker,
    (SUM(runs_off_bat)/COUNT(*))*100 AS batting_strike_rate
FROM deliveries
WHERE ball BETWEEN 15 AND 20
GROUP BY striker
HAVING COUNT(*)>300
ORDER BY batting_strike_rate DESC
LIMIT 10
""").show()

+--------------+-------------------+
|       striker|batting_strike_rate|
+--------------+-------------------+
|AB de Villiers|  215.4555940023068|
|      CH Gayle| 191.11842105263156|
|       RR Pant|  189.4484412470024|
|     SV Samson| 181.23324396782843|
|    AD Russell|  180.4248861911988|
|       V Kohli|  178.8787483702738|
|     RG Sharma|  174.5005875440658|
|  F du Plessis| 173.31288343558282|
|     DA Warner| 173.00275482093664|
|    GJ Maxwell|  172.4852071005917|
+--------------+-------------------+



***35. Which team has the most number of successful chases in IPL history?***

In [39]:
spark.sql('''
SELECT winner,
    SUM(CASE WHEN (toss_decision = 'field' AND toss_winner = winner ) OR 
        (toss_winner <> winner AND toss_decision= 'bat') THEN 1 ELSE 0 END) AS most_win_while_chasing
FROM matches
GROUP BY winner
ORDER BY most_win_while_chasing DESC
''').show(truncate=False)

+---------------------------+----------------------+
|winner                     |most_win_while_chasing|
+---------------------------+----------------------+
|Kolkata Knight Riders      |73                    |
|Mumbai Indians             |69                    |
|Chennai Super Kings        |65                    |
|Royal Challengers Bangalore|61                    |
|Rajasthan Royals           |60                    |
|Kings XI Punjab            |46                    |
|Delhi Daredevils           |42                    |
|Sunrisers Hyderabad        |40                    |
|Delhi Capitals             |22                    |
|Gujarat Titans             |14                    |
|Gujarat Lions              |12                    |
|Deccan Chargers            |11                    |
|Punjab Kings               |8                     |
|Pune Warriors              |6                     |
|Lucknow Super Giants       |5                     |
|Rising Pune Supergiant     |5                

***36. Which team has the highest score chased in IPL history?***

In [56]:
spark.sql('''
WITH inning_scores AS (
    SELECT season,
        match_id,
        innings,
        SUM(runs_off_bat+extras) AS inning_score
    FROM deliveries
    WHERE (innings=2 OR innings=1) AND match_id <> 829813 AND
        match_id IN(SELECT DISTINCT id
                        FROM matches
                        WHERE result <> 'D/L')
    GROUP BY 1,2,3
)
SELECT m.team1,m.team2, m.winner,x.inning_score AS Highest_score_chased 
FROM matches m
JOIN (
    SELECT match_id,inning_score
    FROM inning_scores
    WHERE match_id IN(
    SELECT DISTINCT id
    FROM matches
    WHERE (toss_decision = 'field' AND toss_winner = winner ) OR 
            (toss_winner <> winner AND toss_decision= 'bat')
    )
    ORDER BY inning_score DESC
    LIMIT 1
    ) x ON m.id=x.match_id
''').show(truncate=False)

+---------------+----------------+----------------+--------------------+
|team1          |team2           |winner          |Highest_score_chased|
+---------------+----------------+----------------+--------------------+
|Kings XI Punjab|Rajasthan Royals|Rajasthan Royals|226                 |
+---------------+----------------+----------------+--------------------+



***37. Who has hold the record of getting dismissed on 40s in ipl?***

In [63]:
spark.sql("""
WITH batsman_run AS (
    SELECT match_id,
        striker,
        SUM(runs_off_bat) AS total_runs
    FROM deliveries
    GROUP BY 1,2
    )
SELECT striker,
    COUNT(*) AS most_record_of_getting_dismissed_on_40s
FROM batsman_run
WHERE total_runs BETWEEN 40 AND 49
GROUP BY striker
ORDER BY most_record_of_getting_dismissed_on_40s DESC
""").show()

+--------------+---------------------------------------+
|       striker|most_record_of_getting_dismissed_on_40s|
+--------------+---------------------------------------+
|     RG Sharma|                                     19|
|      S Dhawan|                                     18|
|    KD Karthik|                                     18|
|      MS Dhoni|                                     17|
|     SV Samson|                                     16|
|   BB McCullum|                                     15|
|    RV Uthappa|                                     15|
|    AD Russell|                                     15|
|       V Kohli|                                     15|
|     AT Rayudu|                                     15|
|     YK Pathan|                                     14|
|      CH Gayle|                                     14|
|     DA Warner|                                     14|
|       SS Iyer|                                     13|
|AB de Villiers|               

***38.Which bowler has taken most wickets in a match in ipl?***

In [86]:
spark.sql('''
SELECT match_id,
    bowler,
    COUNT(CASE WHEN wicket_type NOT IN('run out','retired hurt') THEN player_dismissed ELSE NULL END) AS total_wicket,
    SUM(runs_off_bat + COALESCE(wides,0) +COALESCE(noballs,0)) AS runs
FROM deliveries
GROUP BY match_id,bowler
ORDER BY total_wicket DESC
LIMIT 10
''').show(truncate=False)

+--------+--------------+------------+----+
|match_id|bowler        |total_wicket|runs|
+--------+--------------+------------+----+
|1178394 |AS Joseph     |6           |12.0|
|980979  |A Zampa       |6           |19.0|
|336005  |Sohail Tanvir |6           |14.0|
|1370351 |Akash Madhwal |5           |5.0 |
|1254111 |Arshdeep Singh|5           |32.0|
|1304086 |Umran Malik   |5           |25.0|
|1304100 |PWH de Silva  |5           |18.0|
|1359477 |MA Wood       |5           |14.0|
|1216497 |CV Varun      |5           |20.0|
|1254062 |AD Russell    |5           |15.0|
+--------+--------------+------------+----+



***39. Most Runs Conceded By A Bowler in IPL***

In [85]:
spark.sql('''
SELECT bowler,
    SUM(runs_off_bat + COALESCE(wides,0)+COALESCE(noballs,0)) AS runs
    
FROM(
    SELECT *,CAST(ball AS int) AS over
    FROM deliveries
    )
GROUP BY match_id,bowler
ORDER BY runs DESC
''').show(truncate=False)

+-----------------+----+
|bowler           |runs|
+-----------------+----+
|Basil Thampi     |70.0|
|Yash Dayal       |69.0|
|Mujeeb Ur Rahman |66.0|
|Arshdeep Singh   |66.0|
|I Sharma         |66.0|
|Sandeep Sharma   |65.0|
|UT Yadav         |65.0|
|JR Hazlewood     |64.0|
|S Kaul           |64.0|
|M Jansen         |63.0|
|TA Boult         |63.0|
|AB Dinda         |63.0|
|VR Aaron         |63.0|
|Vijaykumar Vyshak|62.0|
|L Ngidi          |62.0|
|MG Neser         |62.0|
|TG Southee       |61.0|
|SR Watson        |61.0|
|Avesh Khan       |60.0|
|T Natarajan      |60.0|
+-----------------+----+
only showing top 20 rows



***40. Most runs in an over in ipl***

In [87]:
spark.sql('''
SELECT bowler,
    SUM(runs_off_bat + COALESCE(wides,0) + COALESCE(noballs,0)) AS runs
    
FROM(
    SELECT *,CAST(ball AS int) AS over
    FROM deliveries
    )
GROUP BY match_id,bowler,over
ORDER BY runs DESC
''').show(truncate=False)

+---------------+----+
|bowler         |runs|
+---------------+----+
|HV Patel       |37.0|
|P Parameswaran |37.0|
|DR Sams        |35.0|
|P Awana        |33.0|
|RS Bopara      |33.0|
|Arjun Tendulkar|31.0|
|Abhishek Sharma|31.0|
|Yash Dayal     |31.0|
|R Sharma       |31.0|
|SS Cottrell    |30.0|
|Shivam Mavi    |30.0|
|CJ Jordan      |30.0|
|OF Smith       |30.0|
|SM Curran      |30.0|
|L Ngidi        |30.0|
|DJ Bravo       |30.0|
|S Kaushik      |30.0|
|JJ van der Wath|30.0|
|A Symonds      |30.0|
|Shivam Mavi    |29.0|
+---------------+----+
only showing top 20 rows



***41. Which team has most 200+ totals by each ipl franchise ?***

In [23]:
spark.sql('''
WITH Total_inning_Runs AS (
SELECT match_id,
    batting_team,
    SUM(runs_off_bat + extras ) AS inning_runs
FROM deliveries
GROUP BY match_id,batting_team
HAVING inning_runs>=200
)
SELECT batting_team,
    COUNT(*) AS most_200_scores
FROM Total_inning_Runs
GROUP BY 1
ORDER BY most_200_scores DESC
''').show(truncate=False)

+---------------------------+---------------+
|batting_team               |most_200_scores|
+---------------------------+---------------+
|Chennai Super Kings        |28             |
|Royal Challengers Bangalore|24             |
|Mumbai Indians             |22             |
|Kolkata Knight Riders      |19             |
|Rajasthan Royals           |18             |
|Sunrisers Hyderabad        |15             |
|Kings XI Punjab            |15             |
|Punjab Kings               |7              |
|Delhi Capitals             |6              |
|Lucknow Super Giants       |5              |
|Gujarat Titans             |5              |
|Delhi Daredevils           |5              |
|Deccan Chargers            |1              |
|Gujarat Lions              |1              |
+---------------------------+---------------+



***42. Which player has Most runs in ipl history?***

In [25]:
spark.sql('''
WITH BatsmanRuns AS (
SELECT match_id,
    striker,
    SUM(runs_off_bat) AS runs
FROM deliveries
GROUP BY match_id,striker
ORDER BY runs DESC
)
SELECT striker,
    SUM(runs) AS most_runs
FROM BatsmanRuns
GROUP BY 1
ORDER BY most_runs DESC
''').show(truncate=False)

+--------------+---------+
|striker       |most_runs|
+--------------+---------+
|V Kohli       |7273     |
|S Dhawan      |6617     |
|DA Warner     |6399     |
|RG Sharma     |6213     |
|SK Raina      |5536     |
|AB de Villiers|5181     |
|MS Dhoni      |5082     |
|CH Gayle      |4997     |
|RV Uthappa    |4954     |
|KD Karthik    |4517     |
|AM Rahane     |4400     |
|AT Rayudu     |4348     |
|G Gambhir     |4217     |
|KL Rahul      |4169     |
|F du Plessis  |4133     |
|SV Samson     |3888     |
|SR Watson     |3880     |
|MK Pandey     |3817     |
|KA Pollard    |3437     |
|SA Yadav      |3249     |
+--------------+---------+
only showing top 20 rows



***42. Which player has Most wickets in ipl history?***

In [ ]:
spark.sql('''
WITH BatsmanRuns AS (
SELECT match_id,
    striker,
    SUM(runs_off_bat) AS runs
FROM deliveries
GROUP BY match_id,striker
ORDER BY runs DESC
)
SELECT striker,
    SUM(runs) AS most_runs
FROM BatsmanRuns
GROUP BY 1
ORDER BY most_runs DESC
''').show(truncate=False)

***43. Which player has played most number of matches for a franchise in ipl?***

In [10]:
# I have solved this question based on the data
spark.sql('''
WITH BatsmanMatchPlayed AS (
        SELECT match_id,
            striker,
            COUNT(DISTINCT match_id) AS total_matches
        FROM deliveries
        GROUP BY match_id,striker
        ORDER BY total_matches DESC
        ),
    BowlerMatchesPlayed AS (
        SELECT match_id,
            striker,
            COUNT(DISTINCT match_id) AS total_matches
        FROM deliveries
        GROUP BY match_id,striker
        ORDER BY total_matches DESC
        )
SELECT striker,
    SUM(total_matches) AS match_played
FROM (
    SELECT *
    FROM BatsmanMatchPlayed
    
    UNION
    
    SELECT *
    FROM BowlerMatchesPlayed
    )
GROUP BY striker
ORDER BY match_played DESC
''').show(truncate=False)

+--------------+------------+
|striker       |match_played|
+--------------+------------+
|RG Sharma     |237         |
|V Kohli       |229         |
|KD Karthik    |220         |
|MS Dhoni      |217         |
|S Dhawan      |216         |
|SK Raina      |200         |
|RV Uthappa    |197         |
|AT Rayudu     |185         |
|DA Warner     |176         |
|AB de Villiers|170         |
|RA Jadeja     |169         |
|KA Pollard    |168         |
|AM Rahane     |159         |
|MK Pandey     |157         |
|YK Pathan     |153         |
|G Gambhir     |151         |
|SV Samson     |147         |
|CH Gayle      |141         |
|SR Watson     |141         |
|PA Patel      |136         |
+--------------+------------+
only showing top 20 rows



In [11]:
spark.sql('''
SELECT *
FROM deliveries
''').show(truncate=False)

+--------+------+----------+--------------------------------+-------+----+--------------+-------------------+------------+------------+------------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+------------+
|match_id|season|start_date|venue                           |innings|ball|batting_team  |bowling_team       |striker     |non_striker |bowler      |runs_off_bat|extras|wides|noballs|byes|legbyes|penalty|wicket_type|player_dismissed|other_wicket_type|other_player_dismissed|cricsheet_id|
+--------+------+----------+--------------------------------+-------+----+--------------+-------------------+------------+------------+------------+------------+------+-----+-------+----+-------+-------+-----------+----------------+-----------------+----------------------+------------+
|1370353 |2023  |2023-05-29|Narendra Modi Stadium, Ahmedabad|1      |0.1 |Gujarat Titans|Chennai Super Kings|WP Saha     |Shubman Gill|DL C

In [36]:
spark.sql('''
DESCRIBE deliveries
''').show(truncate=False)

+----------------+---------+-------+
|col_name        |data_type|comment|
+----------------+---------+-------+
|match_id        |int      |NULL   |
|season          |string   |NULL   |
|start_date      |date     |NULL   |
|venue           |string   |NULL   |
|innings         |int      |NULL   |
|ball            |double   |NULL   |
|batting_team    |string   |NULL   |
|bowling_team    |string   |NULL   |
|striker         |string   |NULL   |
|non_striker     |string   |NULL   |
|bowler          |string   |NULL   |
|runs_off_bat    |int      |NULL   |
|extras          |int      |NULL   |
|wides           |double   |NULL   |
|noballs         |double   |NULL   |
|byes            |double   |NULL   |
|legbyes         |double   |NULL   |
|penalty         |double   |NULL   |
|wicket_type     |string   |NULL   |
|player_dismissed|string   |NULL   |
+----------------+---------+-------+
only showing top 20 rows



In [103]:
# there is a mistake in the data. in  match id 829813 result should be no result but in the data result was given normal.

#spark.sql('''
#UPDATE matches
# SET result='No result'
# WHERE id=829813
# ''').show(truncate=False)

In [40]:
spark.sql('''

''').show(truncate=False)

+----------------------------------------------------------------+------------+
|venue                                                           |average_runs|
+----------------------------------------------------------------+------------+
|Himachal Pradesh Cricket Association Stadium, Dharamsala        |196.75      |
|M Chinnaswamy Stadium, Bengaluru                                |191.57      |
|Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh|185.6       |
|Eden Gardens, Kolkata                                           |185.06      |
|Barsapara Cricket Stadium, Guwahati                             |182.5       |
|Arun Jaitley Stadium, Delhi                                     |175.41      |
|Brabourne Stadium                                               |174.05      |
|Punjab Cricket Association IS Bindra Stadium                    |173.8       |
|Wankhede Stadium, Mumbai                                        |171.55      |
|Brabourne Stadium, Mumbai              

In [50]:
spark.sql('''
SELECT *
FROM matches
ORDER BY season DESC
1370353
''').show(truncate=False)

+-------+------+----------+----------+---------------------------+---------------------------+---------------------------+-------------+------+----------+---------------------------+-----------+--------------+---------------+---------------------------------------------------------------------+---------------------+--------------------+---------------------+
|id     |season|city      |date      |team1                      |team2                      |toss_winner                |toss_decision|result|dl_applied|winner                     |win_by_runs|win_by_wickets|player_of_match|venue                                                                |umpire1              |umpire2             |umpire3              |
+-------+------+----------+----------+---------------------------+---------------------------+---------------------------+-------------+------+----------+---------------------------+-----------+--------------+---------------+-----------------------------------------------------